1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
df = pd.read_csv("Churn_Modelling.csv")
df.drop(columns=["RowNumber", "CustomerId", "Surname"], inplace=True)
df.head(3)

In [ ]:
df.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, :"EstimatedSalary"], df['Exited'], random_state=0)

In [ ]:
X_train.shape, X_test.shape

сделаем пайплайн для логистической регрессии, непрервные признаки стандартизируем, дискретные закодируем через OneHot

In [ ]:
class CreateDf(BaseEstimator, TransformerMixin):
    def __init__(self, col_names):
        self.col_names = col_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        out = X.copy()
        return pd.DataFrame(out, columns=self.col_names)

In [ ]:
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, col_names):
        self.col_names = col_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return pd.get_dummies(X, columns=self.col_names)

In [ ]:
standart_scaler_trans = make_column_transformer(
                                                (StandardScaler(), ["CreditScore", "Age", "Balance", 
                                                                    "NumOfProducts", "EstimatedSalary"]),
                                                remainder="passthrough"
                                                )

In [ ]:
log_res_trans = make_pipeline(standart_scaler_trans,
                              CreateDf(["CreditScore", "Age", "Balance", "NumOfProducts", "EstimatedSalary",
                                        "Geography", "Gender", "Tenure", "HasCrCard", "IsActiveMember"]))

In [ ]:
# посмотрим как отрабатывает трансформер

In [ ]:
log_res_trans.fit_transform(X_train).head(3)

In [ ]:
X_train.head(3)

In [ ]:
#  отрабатывает нормально, фичи не путает, перезапишем его дополнив onehot

In [ ]:
log_res_trans = make_pipeline(standart_scaler_trans,
                              CreateDf(["CreditScore", "Age", "Balance", "NumOfProducts", "EstimatedSalary",
                                        "Geography", "Gender", "Tenure", "HasCrCard", "IsActiveMember"]),
                              OHEEncoder(["Geography", "Gender", "Tenure"]))

In [ ]:
#  не будет кодировать переменные HasCrCard и IsActiveMember потому что они уже являются бинарными

In [ ]:
log_res_trans.fit_transform(X_train).head(3)

In [ ]:
#  чтож вроде закодировалось вполне себе нормально, можно попробовать обучить модель логистической регресии и сделать пайп

In [ ]:
log_res_solver = LogisticRegression(C=0.6, random_state=42)

In [ ]:
final_logres_pipe = make_pipeline(log_res_trans, log_res_solver)

In [ ]:
final_logres_pipe.fit(X_train, y_train)

In [ ]:
logres_predict = final_logres_pipe.predict_proba(X_test)[:, 1]
logres_predict

In [ ]:
from sklearn.metrics import roc_auc_score, precision_recall_curve, confusion_matrix

In [ ]:
def best_thresold(test, preds):
    precision, recall, thresholds = precision_recall_curve(test, preds)

    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                            fscore[ix],
                                                                            precision[ix],
                                                                            recall[ix]))
    return thresholds[ix]

In [ ]:
logres_threshold = best_thresold(y_test, logres_predict)

In [ ]:
roc_auc_score(y_test, logres_predict)

не самый лушчший результат, поскольку мы узнаем чуть больше половины реально уходящих клиентов и при этом совершаем еще ошибку второго рода еще большую 1-46,5% = 53,5%. Т.е ошибаемся в том что это за клиент.  Т.е. мы не можем ни нормально определить уходящих, ни обеспечить понимание в разделении уходящих и нет. И вишенкой на торте roc_auc говорящий о том, что модель вполне себе способная. Это связано как раз с дисбалансом классов и дает такой результат, потому что у нас большая оценка по FN получается

Посотрим что будет, при использовании такой модели

3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты. 

In [ ]:
def estimate_economic(test, predict, threshold):
    conf_matrix = confusion_matrix(test, predict>threshold)
    TN = conf_matrix[0, 0]
    FP = conf_matrix[0, 1]
    TP = conf_matrix[1, 1]
    FN = conf_matrix[1, 0]
    total_revenue = TP * 2
    total_expences = (TP + FP) * 1
    net_income = total_revenue - total_expences
    print("Модель зарабатывает {:,}$ при расходах {:,}$. Общая рентабельность {:%}".format(total_revenue, total_expences, 
                                                                                         net_income / total_expences))
    

In [ ]:
estimate_economic(y_test, logres_predict, logres_threshold)

Как видно модель хронически убыточная и за такую реализацию оторвут руки))  
Улучшим модель за счет Catboost

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
cat_solver = CatBoostClassifier(n_estimators=250, depth=7, loss_function="CrossEntropy", 
                                cat_features=["Geography", "Gender", "Tenure", "HasCrCard", "IsActiveMember"],
                                logging_level="Silent")

In [ ]:
cat_trans = make_pipeline(standart_scaler_trans,
                          CreateDf(["CreditScore", "Age", "Balance", "NumOfProducts", "EstimatedSalary",
                                    "Geography", "Gender", "Tenure", "HasCrCard", "IsActiveMember"]))

In [ ]:
cat_trans.fit_transform(X_train).head(3)

In [ ]:
# вроде трансформер возвращает нормальный датафрейм, едем дальше))

In [ ]:
final_cat_pipe = make_pipeline(cat_trans, cat_solver)

In [ ]:
final_cat_pipe.fit(X_train, y_train)

In [ ]:
cat_predict = final_cat_pipe.predict_proba(X_test)[:, 1]
cat_predict

In [ ]:
cat_threshold = best_thresold(y_test, cat_predict)

In [ ]:
roc_auc_score(y_test, cat_predict)

In [ ]:
estimate_economic(y_test, cat_predict, cat_threshold)

Ну вот совсем уже другое дело)) чисто на одной модели смогли улучшить результат до нормальной рентабельности. При этом как видим roc_auc поднялся всего лишь на 0.1 где-то. Поэтому в слепую применять такую метрику наврядли нужно при решени таких задач классификации, потому что информативность ее для понимания экономики низкая